In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    # Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산 구현
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0])


In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3876, -0.1988,  0.2968,  0.0257, -0.0902, -0.2228,  0.0375, -0.2414,
          0.1309,  0.0188, -0.1610,  0.1516,  0.7776,  0.5499,  0.0398,  0.4402,
          0.9672,  0.1986, -0.1058, -0.0052],
        [ 0.5206, -0.1120,  0.4262,  0.1964, -0.0680, -0.1048,  0.2356, -0.2551,
          0.2130, -0.1736, -0.3627,  0.0682,  0.6777,  0.6465,  0.0884,  0.4442,
          0.7475, -0.2297, -0.2409, -0.0464],
        [ 0.2623, -0.1684,  0.2187,  0.2464, -0.0649, -0.1653,  0.1452, -0.1187,
          0.0229,  0.2814, -0.3190,  0.1868,  0.5769,  0.2573,  0.5544,  0.6906,
          0.2258, -0.0372, -0.0904, -0.1125]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3876, 0.0000, 0.2968, 0.0257, 0.0000, 0.0000, 0.0375, 0.0000, 0.1309,
         0.0188, 0.0000, 0.1516, 0.7776, 0.5499, 0.0398, 0.4402, 0.9672, 0.1986,
         0.0000, 0.0000],
        [0.5206, 0.0000, 0.4262, 0.1964, 0.0000, 0.0000, 0.2356, 0.0000, 0.2130,
         0.0000, 0.0000, 0.0682, 0.6777, 0.6465, 0.08

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [13]:
print(f"Model Structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model Structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0236,  0.0178,  0.0295,  ...,  0.0105,  0.0140, -0.0068],
        [-0.0261,  0.0154, -0.0322,  ...,  0.0002,  0.0299,  0.0104]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0204,  0.0122], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0250, -0.0281,  0.0032,  ...,  0.0377,  0.0123,  0.0163],
        [-0.0395, -0.0437,  0.0230,  ...,  0.0278,  0.0080,  0.0351]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Siz